**NOTE:** This Jupyter Notebook contains the code to train the custom cnn on the training data and analysing its performance.

**Importing the necessary modules**

In [1]:
import numpy as np
import cv2
import os
from os import listdir
import time 
import matplotlib.pyplot as plt 
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle


import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Model, load_model

**Creating functions for loading the data**

In [ ]:
def load_data(dir_list,img_size):
    X=[]
    y=[]
    
    img_width,img_height=img_size
    
    for dir in dir_list:
        if dir.endswith('0'):
            label=0
        elif dir.endswith('1'):
            label=1
        elif dir.endswith('2'):
            label=2
        elif dir.endswith('3'):
            label=3
        elif dir.endswith('4'):
            label=4
        elif dir.endswith('5'):
            label=5
        elif dir.endswith('6'):
            label=6
        elif dir.endswith('7'):
            label=7
        elif dir.endswith('8'):
            label=8
        elif dir.endswith('9'):
            label=9
        elif dir.endswith('plus'):
            label=10
        elif dir.endswith('minus'):
            label=11
        elif dir.endswith('multiply'):
            label=12
        elif dir.endswith('divide'):
            label=13
        elif dir.endswith('equal'):
            label=14
        elif dir.endswith('clear'):
            label=15 
        
            
        for f_name in os.listdir(dir):
            img_path=os.path.join(dir,f_name)
            img=cv2.imread(img_path)
            
            if img is None:
                print(f"[WARN] Skipping unreadable image: {img_path}")
                continue
            
            img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            img=cv2.resize(img,(img_width,img_height))
            img=img.reshape((img_width,img_height,1))
            img=img/255.0 
            
            X.append(img)
            y.append(label)

        
    X=np.array(X)
    y=np.array(y)
    X,y=shuffle(X,y)
            
    print(f"Number of samples: {len(X)}")
    print(f"Dimensions of X: {X.shape}")
    print(f"Dimensions of y: {y.shape}")

    return X,y

**Loading all the image data and converting them into numpy arrays**

In [5]:
aug_path=r"Augmented Data"

paths = [os.path.join(aug_path, str(i)) for i in range(10)]

img_width,img_height=(128,128)

X,y=load_data(paths,(img_width,img_height))

Number of samples: 10240
Dimensions of X: (10240, 128, 128, 1)
Dimensions of y: (10240,)


**Saving X and y as numpy arrays (useful if using colab or kaggle to train)**

In [6]:
np.savez_compressed("data.npz",X=X,y=y)

**Splitting the data into test and train**

In [ ]:
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5)

**Loading the model**

In [ ]:
model=load_model("custom_detection_model.keras")

**Making Adjustments to reduce overfitting and training time**

In [ ]:
checkpoint=ModelCheckpoint(
    "custom_detection_model.keras",
    monitor='val_accuracy',
    save_best_only=True,
    verbose=2,
    mode="max"
)

early_stop=EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    restore_best_weights=True
)

**Training**

In [ ]:
model.fit(x=X_train,y=y_train,batch_size=64,epochs=50,validation_data=(X_val,y_val),callbacks=[checkpoint, early_stop])
